# NPU device properties

In [1]:
import openvino as ov

core = ov.Core()
core.available_devices

['CPU', 'NPU']

In [2]:
import openvino.properties as props


device = "NPU"

core.get_property(device, props.device.full_name)

'Intel(R) AI Boost'

## Get all properties 

In [3]:
print(f"{device} SUPPORTED_PROPERTIES:\n")
supported_properties = core.get_property(device, props.supported_properties)
indent = len(max(supported_properties, key=len))

for property_key in supported_properties:
    if property_key not in ("SUPPORTED_METRICS", "SUPPORTED_CONFIG_KEYS", "SUPPORTED_PROPERTIES"):
        try:
            property_val = core.get_property(device, property_key)
        except TypeError:
            property_val = "UNSUPPORTED TYPE"
        print(f"{property_key:<{indent}}: {property_val}")

NPU SUPPORTED_PROPERTIES:

AVAILABLE_DEVICES               : ['3720']
CACHE_DIR                       : 
COMPILATION_NUM_THREADS         : 20
DEVICE_ARCHITECTURE             : 3720
DEVICE_GOPS                     : {<Type: 'bfloat16'>: 0.0, <Type: 'float16'>: 6553.60009765625, <Type: 'float32'>: 0.0, <Type: 'int8_t'>: 13107.2001953125, <Type: 'uint8_t'>: 13107.2001953125}
DEVICE_ID                       : 
DEVICE_PCI_INFO                 : {domain: 0 bus: 0 device: 0xb function: 0}
DEVICE_TYPE                     : Type.INTEGRATED
DEVICE_UUID                     : 80d1d11eb73811eab3de0242ac130004
ENABLE_CPU_PINNING              : False
EXECUTION_DEVICES               : NPU
EXECUTION_MODE_HINT             : ExecutionMode.PERFORMANCE
FULL_DEVICE_NAME                : Intel(R) AI Boost
INFERENCE_PRECISION_HINT        : <Type: 'float16'>
LOG_LEVEL                       : Level.ERR
MODEL_PRIORITY                  : Priority.MEDIUM
NPU_BYPASS_UMD_CACHING          : False
NPU_COMPILATION_MODE

## Download a model from the hub

In [4]:
import huggingface_hub as hf_hub

from pathlib import Path

# create a directory for resnet model file
MODEL_DIRECTORY_PATH = Path("artifacts")
MODEL_DIRECTORY_PATH.mkdir(exist_ok=True)

model_name = "resnet50"

precision = "FP16"

model_path = MODEL_DIRECTORY_PATH / "ir_model" / f"{model_name}_{precision.lower()}.xml"

model = None
if not model_path.exists():
    hf_hub.snapshot_download("katuni4ka/resnet50_fp16", local_dir=model_path.parent)
    print("IR model saved to {}".format(model_path))
    model = core.read_model(model_path)
else:
    print("Read IR model from {}".format(model_path))
    model = core.read_model(model_path)

Read IR model from artifacts\ir_model\resnet50_fp16.xml


In [5]:
compiled_model = core.compile_model(model, device)
compiled_model

<CompiledModel:
inputs[
<ConstOutput: names[x] shape[1,3,224,224] type: f32>
]
outputs[
<ConstOutput: names[x.45] shape[1,1000] type: f32>
]>

## Performance hints

In [6]:
import openvino.properties.hint as hints


compiled_model = core.compile_model(model, device, {hints.performance_mode(): hints.PerformanceMode.LATENCY})

# compiled_model = core.compile_model(model, device, {hints.performance_mode(): hints.PerformanceMode.THROUGHPUT})